In [ ]:
!pip install langchain

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00


In [ ]:
import os

In [ ]:
os.environ['OPENAI_API_KEY'] = 'Add API Key HERE'

In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.6)

name = llm('Suggest me a name for Indian resturant')

print(name)



Maya Masala


In [ ]:
from langchain import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = 'Suggest me a name for {cuisine} resturant'
)

prompt_template_name.format(cuisine = 'Italian')

'Suggest me a name for Italian resturant'

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)

chain.run('italian')

"\n\nMamma Mia's Italian Kitchen"

## SimpleSequentialChain

In [ ]:
prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = 'Suggest me a name for {cuisine} resturant'
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_name = PromptTemplate(
    input_variables = ['resturant_name'],
    template = 'Suggest me some food items for {resturant_name} resturant. Return it as a comma separated list'
)

item_chain = LLMChain(llm=llm, prompt=prompt_template_name)

In [ ]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains=[name_chain,item_chain])

chain.run('Indian')

'\n\nTandoori Chicken, Masala Dosa, Samosas, Biryani, Naan, Palak Paneer, Chole, Kebabs, Vindaloo, Tikka Masala.'

## SequentialChain

In [ ]:
prompt_template_name = PromptTemplate(
    input_variables = ['cuisine'],
    template = 'Suggest me a name for {cuisine} resturant'
)

name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key='resturant_name')

prompt_template_name = PromptTemplate(
    input_variables = ['resturant_name'],
    template = 'Suggest me some food items for {resturant_name} resturant. Return it as a comma separated list'
)

item_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key='menu_items')

In [ ]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains=[name_chain,item_chain],
    input_variables=['cuisine'],
    output_variables=['resturant_name','menu_items']
)

chain({'cuisine':'Indian'})

{'cuisine': 'Indian',
 'resturant_name': '\n\nKashmiri Khana',
 'menu_items': '\n\nRoghan Josh, Yakhni Pulao, Tabak Maaz, Kashmiri Dum Aloo, Nadru Yakhni, Kashmiri Pulao, Kashmiri Muji Gaad, Gushtaba, Haak, Kashmiri Biryani'}

## Agents

In [ ]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=4ec406137a3fc528912f355c12d6583cf2338be6bcd6c32b198cb0da6bc68026
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

tools = load_tools(['wikipedia','llm-math'],llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run('What is the GDP of India in 2022 and also add 5 into it')




> Entering new AgentExecutor chain...
 I need to find out what India's GDP is and then add 5 to it
Action: Wikipedia
Action Input: GDP of India
Observation: Page: List of Indian states and union territories by GDP
Summary: These are lists of Indian states and union territories by their nominal gross state domestic product (GSDP). GSDP is the sum of all value added by industries within each state or union territory and serves as a counterpart to the national gross domestic product (GDP).In India, the Government accounts for around 21% of the GDP, agriculture accounts for 21%, the corporate sector accounts for 12% and the balance 48% of the GDP is sourced from small proprietorship and partnership companies, unorganized sector and households.
The following list gives the latest available gross state domestic product (GSDP) figures for all Indian States and Union Territories at current prices in millions or trillions of Indian rupees. No data is available for the union territories of Dad

'The GDP of India in 2022 is estimated to be around 4.76 trillion US dollars. Adding 5 to this, the GDP of India in 2022 is estimated to be around 4.81 trillion US dollars.'

## Memory

In [ ]:
chain({'cuisine':'Indian'})

{'cuisine': 'Indian',
 'resturant_name': '\n\nMaharaja Palace',
 'menu_items': '\n\nTandoori Chicken, Butter Chicken, Paneer Tikka Masala, Seekh Kebab, Dal Makhani, Naan, Kulfi, Gulab Jamun.'}

In [ ]:
chain({'cuisine':'French'})

{'cuisine': 'French',
 'resturant_name': '\n\nLe Bistro Francais',
 'menu_items': '\n\nSteak au Poivre, Croque Monsieur, Coq au Vin, Boeuf Bourguignon, Cassoulet, Ratatouille, Quiche Lorraine, Escargots, Steak Frites, Bouillabaisse.'}

In [ ]:
print(chain.memory) #by default it has no memory

None


In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_name, memory=memory)

chain.run('italian')

'\n\nPizza, Lasagna, Spaghetti, Focaccia Bread, Ravioli, Bruschetta, Caprese Salad, Gnocchi, Risotto, Calzone, Tortellini, Antipasto Platter.'

In [ ]:
print(chain.memory)

chat_memory=ChatMessageHistory(messages=[HumanMessage(content='italian', additional_kwargs={}, example=False), AIMessage(content='\n\nPizza, Lasagna, Spaghetti, Focaccia Bread, Ravioli, Bruschetta, Caprese Salad, Gnocchi, Risotto, Calzone, Tortellini, Antipasto Platter.', additional_kwargs={}, example=False)]) output_key=None input_key=None return_messages=False human_prefix='Human' ai_prefix='AI' memory_key='history'


In [ ]:
print(chain.memory.buffer)

Human: italian
AI: 

Pizza, Lasagna, Spaghetti, Focaccia Bread, Ravioli, Bruschetta, Caprese Salad, Gnocchi, Risotto, Calzone, Tortellini, Antipasto Platter.


In [ ]:
from langchain.chains import ConversationChain

convo = ConversationChain(
    llm = OpenAI(temperature=0.7)
)

print(convo.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [ ]:
convo.run("Who won the football world cup in 2010?")

' The 2010 FIFA World Cup was won by Spain.'

In [ ]:
convo.run("What is 5+5?")

' 10.'

In [ ]:
convo.run("Who was the caption?")

' The captain of the Spanish team that won the 2010 FIFA World Cup was Iker Casillas.'

In [ ]:
print(convo.memory.buffer)

Human: Who won the football world cup in 2010?
AI:  The 2010 FIFA World Cup was won by Spain.
Human: What is 5+5?
AI:  10.
Human: Who was the caption?
AI:  The captain of the Spanish team that won the 2010 FIFA World Cup was Iker Casillas.
